In [70]:
import pandas as pd
import re
import sys

In [71]:
#Modulo que  agrega el path de los modulos al sys.path
from pathlib import Path

# Obtener la ruta al directorio "modulos"
ruta_modulos = Path("../modulos").resolve()

# Agregar la ruta al sys.path
if str(ruta_modulos) not in sys.path:
    sys.path.append(str(ruta_modulos))          

# Obtener la ruta al directorio "modulos"
ruta_modulos = Path("../modulos").resolve()
print(f"Ruta de modulos: {ruta_modulos}")  # Verifica la ruta que se obtiene


Ruta de modulos: C:\Users\OPERADOR\Desktop\PROYECTO DIGITALIZACION\digitalizacion_distriapp\backend\src\modulos


In [72]:
#MODULE DATA LOADER 

# Carga de datos
def load_data(df_archivo):
    """
    Carga un archivo CSV en un DataFrame de pandas.

    Parámetros:
        archivo (str): El nombre o la ruta del archivo CSV.

    Retorna:
        DataFrame: El DataFrame que contiene los datos del archivo CSV.
    """
    try:
        # Cargar el archivo CSV en un DataFrame
        df = pd.read_csv(df_archivo)
        return df
    except Exception as e:
        print(f"Error al cargar el archivo {df_archivo}: {e}")
        return None

# Variables del DF
name_df_provedor = "prueba.csv" 
name_df_productos =  "L_STMaestroPrd.csv"     

#Variables Funcion  lodad_Data
df_proveedor = load_data(name_df_provedor)
df_productos = load_data(name_df_productos)

#Copias del DataFrame 
copy_dfProductos = df_productos
copy_dfProveedores = df_proveedor

C:\Users\OPERADOR\AppData\Local\Temp\ipykernel_16712\2906993674.py:16: DtypeWarning: Columns (7,14,17,18,21,28,39,42,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df_archivo)


In [73]:
# Module: SEARCH UTILS

def laboratorio_input_df(df, columna):
    """
    Busca coincidencias parciales en una columna de un DataFrame basado en una entrada del usuario.

    Parámetros:
        df (DataFrame): El DataFrame en el que se buscarán las coincidencias.
        columna (str): El nombre de la columna en la que se realizará la búsqueda.

    Solicita:
        str: Nombre del laboratorio ingresado por el usuario.

    Retorna:
        DataFrame: Un DataFrame con las coincidencias encontradas.
        None: Si no se encuentran coincidencias.

    Ejemplo de uso:
        nombre_columna = 'atrib0'
        laboratorio_coincidencias = laboratorio_input_df(copy_dfProductos, nombre_columna)
    """
    # Solicitar el nombre del laboratorio al usuario
    input_name = input("Ingrese el nombre del laboratorio: ").strip()

    # Buscar coincidencias parciales en la columna especificada
    tablas_coincidencias = df[df[columna].str.contains(input_name, case=False, na=False)]

    # Verificar y retornar los resultados
    if not tablas_coincidencias.empty:
        print("Se encontraron valores coincidentes:")
        return tablas_coincidencias
    else:
        print("No se encontraron coincidencias.")
        return None


#Siempre se le manda la copia del DF
columna_nombre = 'atrib0'

# Búsqueda de coincidencias en el DataFrame

# Buscar coincidencias en el DataFrame de productos según el nombre del laboratorio proporcionado
laboratorio_coincidencias_df = laboratorio_input_df(
    copy_dfProductos,  # Copia del DataFrame de productos
    columna_nombre  # Nombre de la columna a buscar
)

# Imprimir DF DEL LABORATORIO COINCIDENTE
# Se debe usar este data frame para hacer las coincidencias con la lista de proveedores
# que envia el proveedor.  
laboratorio_coincidencias_df  



Se encontraron valores coincidentes:


,niprod,cod_prod,cod_abrev,descrip1,descrip2,abrev,cod_catprod,codbarra,cod_fbarra,unid_v,...,atrib0,atrib1,atrib2,atrib3,atrib4,atrib5,atrib6,atrib7,atrib8,atrib9
3937,105921.0,36302,36302,ANTIHEMORROIDAL SERTEX pda.x 15 g,NaN,ANTIHEMORROIDAL SERT,PT,7795228001276,NaN,UN,...,SERTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5939,105922.0,36671,36671,BABY-TEX pda.x 50 g,NaN,BABY-TEX pda.x 50 g,PTO,7795228001306,NaN,UN,...,SERTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9330,105923.0,33247,33247,BUCLORHEX fco.x 500 ml,NaN,BUCLORHEX fco.x 500,PTO,7795228001207,NaN,UN,...,SERTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16078,105924.0,37178,37178,DERCOTEX cr.x 15 g,NaN,DERCOTEX cr.x 15 g,PT,7795228001290,NaN,UN,...,SERTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16079,105925.0,36131,36131,DERCOTEX cr.x 30 g,NaN,DERCOTEX cr.x 30 g,PT,7795228001269,NaN,UN,...,SERTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24648,105926.0,28611,28611,GEL ANTIINFLAMATORIO SERTEX pomo x 50 g,NaN,GEL ANTIINFLAMATORIO,PT,NaN,NaN,UN,...,SERTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25584,127932.0,SOA-GLU-01701,GLUTARAL,GLUTARALDEHIDO 2% CON SURFACTANTE BIDON X 4000...,BIDON X 5 LITROS - UM: BIDON,GLUTARAL,MP,NaN,NaN,UN,...,SERTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28094,105927.0,38534,38534,IBUPROFEX 2% susp.oral.x 90 ml,NaN,IBUPROFEX 2% susp.or,PT,7795228001337,NaN,UN,...,SERTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29599,105928.0,24242,24242,JABON NEUTRO SERTEX fco.c/dosific.x 250 ml,NaN,JABON NEUTRO SERTEX,PT,7795228001092,NaN,UN,...,SERTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30827,105929.0,43715,43715,LACLORHEX antiseptico azul x 250ml,NaN,LACLORHEX antiseptic,PT,7795228001108,NaN,UN,...,SERTEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
#MODULO data_normalization: Enfoque de estandarización de datos.

#Funcion REGEX
def clear_column(column):
    """
    Limpia y normaliza el texto en una columna para eliminar términos irrelevantes y uniformar el formato.

    Esta función utiliza expresiones regulares para eliminar números, unidades de medida, abreviaturas comunes, 
    caracteres especiales y palabras redundantes. También convierte el texto a minúsculas, elimina espacios en 
    blanco adicionales y normaliza el formato para facilitar su análisis posterior.

    Parámetros:
        column (str): El texto de la columna a limpiar.

    Retorna:
        str: El texto limpio y normalizado.

    Pasos de limpieza:
        1. Elimina números, unidades de medida y términos irrelevantes (e.g., "mg", "blister", "ml").
        2. Elimina caracteres especiales como dos puntos, puntos, comas, guiones, etc.
        3. Elimina espacios en blanco múltiples y normaliza el texto.
        4. Convierte el texto a minúsculas para uniformidad.
        5. Elimina palabras vacías comunes (e.g., "de", "la", "el").
        6. Elimina palabras repetidas consecutivas (e.g., "producto producto").
        7. Elimina términos relacionados con la dosis o presentación (e.g., "20mg", "30ml").

    Ejemplo de uso:
        texto_limpio = clear_column("20mg Blister de 10 unidades: Marca X")
        print(texto_limpio)  # Output: "marca x"
    """
    # Expresión regular para eliminar números, unidades, abreviaturas y términos irrelevantes
    columna_limpia = re.sub(
        r'\b(\d+|mg|comp\.?|blisters?|estuche|x|ml|unidades|marca|botella|caja|referencia|cm|g|m|f\.a\.x|'
        r'jbe\.|sol\.|env\.|lata|vial|pvo\.|sobres|gotero|cápsulas|frascos?|blister|bot\.|mcg|sp|p\.b\.)\b', 
        '', 
        column, 
        flags=re.IGNORECASE
    )
    
    # Eliminar caracteres especiales como los dos puntos, puntos finales, guiones, comas, etc.
    columna_limpia = re.sub(r'[:\.\-,;()]', '', columna_limpia)

    # Eliminar múltiples espacios en blanco y normalizar el texto
    columna_limpia = re.sub(r'\s+', ' ', columna_limpia).strip()

    # Convertir todo el texto a minúsculas
    columna_limpia = columna_limpia.lower()

    # Eliminar palabras vacías comunes
    columna_limpia = re.sub(r'\b(?:de|la|el|en|a|y|con|para|del|sobre|por|medicamento|tratamiento|ac)\b', '', columna_limpia).strip()

    # Eliminar palabras repetidas consecutivas
    columna_limpia = re.sub(r'\b(\w+)\s+\1\b', r'\1', columna_limpia)

    # Eliminar términos relacionados con dosis o presentación
    columna_limpia = re.sub(r'\b(\d+mg|\d+g|\d+ml)\b', '', columna_limpia)

    return columna_limpia

In [75]:
#MODULE DataFrame_Column_Matcher

# Función para obtener coincidencias entre las columnas de dos DataFrames y agregar una columna 'match'
def obtener_coincidencias_con_match(productos_df, proveedores_df, col_productos, col_proveedores):
    """
    Busca coincidencias entre las columnas especificadas de dos DataFrames y agrega una columna 'match' que 
    indica si hubo una coincidencia o no. Si se encuentra una coincidencia, combina los datos relevantes 
    de ambos DataFrames.

    Parámetros:
        productos_df (DataFrame): DataFrame que contiene la información de los productos.
        proveedores_df (DataFrame): DataFrame que contiene la información de los proveedores.
        col_productos (str): Nombre de la columna en el DataFrame de productos que será comparada.
        col_proveedores (str): Nombre de la columna en el DataFrame de proveedores que será comparada.

    Retorna:
        DataFrame: Un DataFrame resultante que incluye:
            - Columnas combinadas de ambos DataFrames en caso de coincidencia.
            - Una columna 'match' que indica True si hay coincidencia, False en caso contrario.

    Pasos del proceso:
        1. Se limpian las columnas relevantes de ambos DataFrames usando la función `clear_column`.
        2. Se iteran las filas del DataFrame de proveedores y se buscan coincidencias en el DataFrame de productos.
        3. Si hay coincidencias, se combinan las filas correspondientes de ambos DataFrames.
        4. Si no hay coincidencias, se agrega la fila del proveedor con valores nulos para las columnas de productos.
    """
    # Crear copias explícitas de los DataFrames para evitar problemas con modificaciones directas
    productos_df = productos_df.copy()
    proveedores_df = proveedores_df.copy()

    # Limpiar las columnas antes de realizar comparaciones
    productos_df[col_productos] = productos_df[col_productos].apply(lambda x: clear_column(str(x)))
    proveedores_df[col_proveedores] = proveedores_df[col_proveedores].apply(lambda x: clear_column(str(x)))

    # Lista para almacenar los resultados
    resultados = []

    # Iterar sobre las filas del DataFrame de proveedores
    for _, proveedor in proveedores_df.iterrows():
        descripcion_proveedor = proveedor[col_proveedores]

        # Buscar coincidencias en el DataFrame de productos
        coincidencias = productos_df[
            productos_df[col_productos].str.contains(descripcion_proveedor, case=False, na=False)
        ]

        if not coincidencias.empty:
            # Si hay coincidencias, combinar las filas relevantes
            for _, producto in coincidencias.iterrows():
                resultado = {**proveedor.to_dict(), **producto.to_dict(), "match": True}
                resultados.append(resultado)
        else:
            # Si no hay coincidencias, agregar la fila del proveedor con valores nulos para los productos
            resultado = proveedor.to_dict()
            for col in productos_df.columns:
                resultado[col] = None
            resultado["match"] = False
            resultados.append(resultado)

    # Convertir la lista de resultados a un DataFrame
    return pd.DataFrame(resultados)

# Ejemplo de uso
resultado_df = obtener_coincidencias_con_match(
    productos_df=laboratorio_coincidencias_df,
    proveedores_df=copy_dfProveedores,
    col_productos="descrip1",
    col_proveedores="DESCRIPCIÓN"
)
# Eliminar las columnas 'atrib5', 'atrib6', 'atrib7', 'atrib8', 'atrib9'
resultado_df = resultado_df.drop(columns=['atrib5', 'atrib6', 'atrib7', 'atrib8', 'atrib9'])
# Mostrar el DataFrame resultante

In [76]:
resultado_df

,EAN,DESCRIPCIÓN,PVP,DROGUERIA,DTO.,CANT.X BULTO,niprod,cod_prod,cod_abrev,descrip1,...,cat_item,cuf_rep,cant_rep,pto_rep,atrib0,atrib1,atrib2,atrib3,atrib4,match
0,7791939000089,aspirina,"$4.904,38","$2.305,06",53%,50,None,None,None,None,...,None,None,None,None,None,None,None,None,None,False
1,7791939001116,cardio vent3 masticables,"$1,31",$393,70%,50,None,None,None,None,...,None,None,None,None,None,None,None,None,None,False
2,7791939000096,calcio base masticables,"$12.995,10","$6.107,70",53%,90,None,None,None,None,...,None,None,None,None,None,None,None,None,None,False
3,7791939000133,carbon vegetal activado,"$14.071,69","$6.613,69",53%,50,None,None,None,None,...,None,None,None,None,None,None,None,None,None,False
4,7791939001383,clonazepam sl,"$ 6.208,60","$ 1.738,41",72%,40,None,None,None,None,...,None,None,None,None,None,None,None,None,None,False
5,7791939001390,clonazepam,"$ 11.824,64","$ 3.310,90",72%,40,None,None,None,None,...,None,None,None,None,None,None,None,None,None,False
6,7791939001413,clonazepam,"$ 11.326,00","$ 3.171,28",72%,40,None,None,None,None,...,None,None,None,None,None,None,None,None,None,False
7,7791939001437,clonazepam,"$ 12.593,93","$ 3.526,30",72%,40,None,None,None,None,...,None,None,None,None,None,None,None,None,None,False
8,7791939000416,ibuprofeno,"$3.100,34","$868,10",72%,50,None,None,None,None,...,None,None,None,None,None,None,None,None,None,False
9,7791939000522,ibuprofeno,"$6.567,98","$1.839,03",72%,40,None,None,None,None,...,None,None,None,None,None,None,None,None,None,False


In [77]:
resultado_df['atrib0']

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
Name: atrib0, dtype: object

In [78]:
#MODULE DATAFRAME QUERY HANDLER
# Solicitar input del laboratorio
#input_laboratorio = input("Ingrese el nombre del laboratorio: ")

# Filtrar los productos que coinciden parcialmente con el laboratorio ingresado
resultados = df_productos[df_productos["atrib0"].str.contains(input_laboratorio, case=False, na=False)]
resultados




# Recuento de laboratorios por nombre de laboratorio (atrib0) y filtrado para "VENT 3"
# Agrupar los datos de 'copy_dfProductos' por la columna "atrib0" (nombre del laboratorio)
# y contar la cantidad de filas por cada valor único en esta columna.
recuento_laboratorios = copy_dfProductos.groupby(copy_dfProductos["atrib0"]).size().reset_index(name='Cantidad de filas')

# Filtrar los resultados para obtener solo las filas donde el nombre del laboratorio es "VENT 3"
result = recuento_laboratorios[recuento_laboratorios["atrib0"] == "ELEA"]
result



,atrib0,Cantidad de filas
304,ELEA,839
